# Evaluation of RAG Using Ragas

In the following notebook we'll explore how to evaluate RAG pipelines using a powerful open-source tool called "Ragas". This will give us tools to evaluate component-wise metrics, as well as end-to-end metrics about the performance of our RAG pipelines.

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room Part #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating a simple RAG pipeline with [LangChain v0.2.0](https://python.langchain.com/v0.2/docs/versions/v0_2/)
  4. Synthetic Dataset Generation for Evaluation using the [Ragas](https://github.com/explodinggradients/ragas) framework.
  

- 🤝 Breakout Room Part #2:
  1. Evaluating our pipeline with Ragas
  3. Making Adjustments to our RAG Pipeline
  4. Evaluating our Adjusted pipeline against our baseline
  5. Testing OpenAI's Claim

The only way to get started is to get started - so let's grab our dependencies for the day!

> NOTE: Using this notebook as presented will occur a charge of ~$3USD from OpenAI usage. Most of this cost is produced by the Synthetic Data Generation step - if you want to reduce costs, please use the provided commented code to leverage `GPT-3.5-Turbo` as the `critic_llm`!

## Motivation

A claim, made by OpenAI, is that their `text-embedding-3-small` is better (generally) than their `text-embedding-ada-002` model.

Here's some passages from their [blog](https://openai.com/blog/new-embedding-models-and-api-updates) about the `text-embedding-3` release:

> `text-embedding-3-small` is our new highly efficient embedding model and provides a significant upgrade over its predecessor, the `text-embedding-ada-002` model...

> **Stronger performance.** Comparing `text-embedding-ada-002` to `text-embedding-3-small`, the average score on a commonly used benchmark for multi-language retrieval ([MIRACL](https://github.com/project-miracl/miracl)) has increased from 31.4% to 44.0%, while the average score on a commonly used benchmark for English tasks ([MTEB](https://github.com/embeddings-benchmark/mteb)) has increased from 61.0% to 62.3%.

Well, with a library like Ragas - we can put that claim to the test!

If what they claim is true - we should see an increase on related metrics by using the new embedding model!

# 🤝 Breakout Room Part #1

## Task 1: Installing Required Libraries

A reminder that one of the [key features](https://blog.langchain.dev/langchain-v0-1-0/) of LangChain v0.1.0 is the compartmentalization of the various LangChain ecosystem packages!

So let's begin grabbing all of our LangChain related packages!

In [8]:
%pip install -U -q langchain langchain-openai langchain_core langchain-community langchainhub openai

Note: you may need to restart the kernel to use updated packages.


We'll also get the "star of the show" today, which is Ragas!

In [9]:
%pip install -qU ragas

Note: you may need to restart the kernel to use updated packages.


We'll be leveraging [QDrant](https://qdrant.tech/) again as our LangChain `VectorStore`.

We'll also install `pymupdf` and its dependencies which will allow us to load PDFs using the `PyMuPDFLoader` in the `langchain-community` package!

In [10]:
%pip install -qU qdrant-client pymupdf pandas

Note: you may need to restart the kernel to use updated packages.


## Task 2: Set Environment Variables

Let's set up our OpenAI API key so we can leverage their API later on.

In [11]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

## Task 3: Creating a Simple RAG Pipeline with LangChain v0.1.0

Building on what we learned last week, we'll be leveraging LangChain v0.1.0 and LCEL to build a simple RAG pipeline that we can baseline with Ragas.

## Building our RAG pipeline

Let's review the basic steps of RAG again:

- Create an Index
- Use retrieval to obtain pieces of context from our Index that are similar to our query
- Use a LLM to generate responses based on the retrieved context

Let's get started by creating our index.

> NOTE: We're going to start leaning on the term "index" to refer to our `VectorStore`, `VectorDatabase`, etc. We can think of "index" as the catch-all term, whereas `VectorStore` and the like relate to the specific technologies used to create, store, and interact with the index.

### Creating an Index

You'll notice that the largest changes (outside of some import changes) are that our old favourite chains are back to being bundled in an easily usable abstraction.

We can still create custom chains using LCEL - but we can also be more confident that our pre-packaged chains are creating using LCEL under the hood.

#### Loading Data

Let's start by loading some data!

> NOTE: You'll notice that we're using a document loader from the community package of LangChain. This is part of the v0.2.0 changes that make the base (`langchain-core`) package remain lightweight while still providing access to some of the more powerful community integrations.

In [12]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(
    "https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf",
)

documents = loader.load()

In [13]:
documents[0].metadata

{'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf',
 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf',
 'page': 0,
 'total_pages': 195,
 'format': 'PDF 1.3',
 'title': 'The Pmarca Blog Archives',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': '',
 'producer': 'Mac OS X 10.10 Quartz PDFContext',
 'creationDate': "D:20150110020418Z00'00'",
 'modDate': "D:20150110020418Z00'00'",
 'trapped': ''}

#### Transforming Data

Now that we've got our single document - let's split it into smaller pieces so we can more effectively leverage it with our retrieval chain!

We'll start with the classic: `RecursiveCharacterTextSplitter`.

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 150,
    chunk_overlap = 50
)

documents = text_splitter.split_documents(documents)

Let's confirm we've split our document.

In [15]:
len(documents)

2636

#### Loading OpenAI Embeddings Model

We'll need a process by which we can convert our text into vectors that allow us to compare to our query vector.

Let's use OpenAI's `text-embedding-ada-002` for this task!

In [16]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

#### Creating a QDrant VectorStore

Now that we have documents - we'll need a place to store them alongside their embeddings.

In [17]:
from langchain_community.vectorstores import Qdrant

qdrant_vector_store = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="PMarca Blogs",
)

#### ❓ Question #1:

List out a few of the techniques that Qdrant uses that make it performant.

One general advantage that Qdrant has is its use of Rust, which is a particularly performant language. One of the more significant performance improvements comes from Qdrant's support for sparse vectors, which reduces the amount of stored vector data that needs to be generated to maintain a high fidelity representation of the source data. Additionally, a recent release introduced a new approach for indexing text data that removes the need for re-indexing immutable text fields. 

#### Creating a Retriever

To complete our index, all that's left to do is expose our vectorstore as a retriever - which we can do the same way we would in previous version of LangChain!

In [18]:
retriever = qdrant_vector_store.as_retriever()

print(retriever)

tags=['Qdrant', 'OpenAIEmbeddings'] vectorstore=<langchain_community.vectorstores.qdrant.Qdrant object at 0x10fa8fc50>


#### Testing our Retriever

Now that we've gone through the trouble of creating our retriever - let's see it in action!

In [19]:
retrieved_documents = retriever.invoke("What is a rule of thumb for selecting an industry to invest in?")

In [20]:
for doc in retrieved_documents:
  print(doc)

page_content='Part 3: Where to go and why
When picking an industry to enter, my favorite rule of thumb is
this:' metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page': 124, 'total_pages': 195, 'format': 'PDF 1.3', 'title': 'The Pmarca Blog Archives', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Mac OS X 10.10 Quartz PDFContext', 'creationDate': "D:20150110020418Z00'00'", 'modDate': "D:20150110020418Z00'00'", 'trapped': '', '_id': '22e9f53c9d7f4f31be39f3267c48f093', '_collection_name': 'PMarca Blogs'}
page_content='have a reasonable chance at succeeding.
Second rule of thumb:
Once you have picked an industry, get right to the center of it' metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2

### Creating a RAG Chain

Now that we have the "R" in RAG taken care of - let's look at creating the "AG"!

#### Creating a Prompt Template

There are a few different ways we could create our prompt template - we could create a custom template, as seen in the code below, or we could simply pull a prompt from the prompt hub! Let's look at an example of that!

In [21]:
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

print(retrieval_qa_prompt)

input_variables=['context', 'input'] optional_variables=['chat_history'] input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} partial_variables={'chat_history': []} metadata={'lc_hub_owner': 'langchain-ai', 'lc_hub_repo': 'retrieval-qa-chat', 'lc_hub_commit_hash': 'b60afb6297176b022244feb83066e10ecadcda7b90423654c4a9d45e7a73cebc'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='Answer any use questions based solely on the context below:\n\n<context>\n{context}\n</context>')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))]


In [22]:
print(retrieval_qa_prompt.messages[0].prompt.template)

Answer any use questions based solely on the context below:

<context>
{context}
</context>


As you can see - the prompt template is simple (and has a small error) - so we'll create our own to be a bit more specific!

In [23]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

#### Setting Up our Basic QA Chain

Now we can instantiate our basic RAG chain!

We'll use LCEL directly just to see an example of it - but you could just as easily use an abstraction here to achieve the same goal!

We'll also ensure to pass-through our context - which is critical for RAGAS.

In [24]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

#### 🏗️ Activity #1:

Describe the pipeline shown above in simple terms. You can include a diagram if desired.

Here's the flow of what's happening in the code block above:
We're setting the LLM that should be used for this pipeline to be OpenAI's ChatGPT 3.5 Turbo. Next, we're setting up our RAG Q+A chain to take in and process the following information:
- The question text is provided as part of the "question" context
- The retriever is told to use that question as its context for searching for similar entities within the index
- The question and the retriever are then collectively set as context for when the LLM is prompted

Let's test it out!

In [25]:
question = "What is a rule of thumb for selecting an industry to invest in?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

The rule of thumb for selecting an industry to invest in is to pick an industry where you have a reasonable chance at succeeding.


In [26]:
question = "What did Pink Floyd have to say about how to proceed when investing in a new industry?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)
print(result["context"])

I don't know.
[Document(metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page': 17, 'total_pages': 195, 'format': 'PDF 1.3', 'title': 'The Pmarca Blog Archives', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Mac OS X 10.10 Quartz PDFContext', 'creationDate': "D:20150110020418Z00'00'", 'modDate': "D:20150110020418Z00'00'", 'trapped': '', '_id': '36b906ae077b4606bb9b2ac5611da7ca', '_collection_name': 'PMarca Blogs'}, page_content='other startups, and also diWerentiated from any large incum-\nbents?\nTiming risk — is it too early? Is it too late?'), Document(metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page': 124, 'total_pages': 195, 'format': 'PDF 1

We can already see that there are some improvements we could make here.

For now, let's switch gears to RAGAS to see how we can leverage that tool to provide us insight into how our pipeline is performing!

## Task 4: Synthetic Dataset Generation for Evaluation using Ragas

Ragas is a powerful library that lets us evaluate our RAG pipeline by collecting input/output/context triplets and obtaining metrics relating to a number of different aspects of our RAG pipeline.

We'll be evaluating on every core metric today, but in order to do that - we'll need to create a test set. Luckily for us, Ragas can do that directly!

### Synthetic Test Set Generation

We can leverage Ragas' [`Synthetic Test Data generation`](https://docs.ragas.io/en/stable/concepts/testset_generation.html) functionality to generate our own synthetic QC pairs - as well as a synthetic ground truth - quite easily!

In [29]:
loader = PyMuPDFLoader(
    "https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf",
)

eval_documents = loader.load()

text_splitter_eval = RecursiveCharacterTextSplitter(
    chunk_size = 1200,
    chunk_overlap = 50
)

eval_documents = text_splitter_eval.split_documents(eval_documents)

#### ❓ Question #2:

##### Why is it important to split our documents using different parameters when creating our synthetic data?
This creates greater potential for variability in the resulting set of test data.

In [30]:
len(eval_documents)

357


> NOTE: 🛑 Using this notebook as presented will occur a charge of ~$3USD from OpenAI usage. Most of this cost is produced by the Synthetic Data Generation step - if you want to reduce costs, please use the provided commented code to leverage GPT-3.5-Turbo as the critic_llm. If you're attempting to create a lot of samples please be aware of cost, as well as rate limits. 🛑

In [32]:
import nest_asyncio
import asyncio
nest_asyncio.apply()

from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

generator_llm = ChatOpenAI(model="gpt-3.5-turbo")
# critic_llm = ChatOpenAI(model="gpt-3.5-turbo") # <--- If you don't have GPT-4 access, or to reduce cost/rate limiting issues.
critic_llm = ChatOpenAI(model="gpt-4o")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

testset = generator.generate_with_langchain_docs(eval_documents, 20, distributions, is_async = False)
testset.to_pandas()

embedding nodes:   0%|          | 0/714 [00:00<?, ?it/s]

Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/20 [00:00<?, ?it/s]

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,How do the roles and responsibilities of execu...,[from scratch. This is a sharp diWerence from ...,The roles and responsibilities of executives i...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
1,How can internal confusion impact a startup's ...,"[developing a large market, as opposed to Xght...",Internal confusion can impact a startup's stra...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
2,How should founders approach market projection...,[Rethink this one from the ground up. Lots of ...,Founders should avoid saying they have no comp...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
3,What is Rachleff's Law of Startup Success and ...,"[And when you have a great market, the team is...",Rachleff's Law of Startup Success states that ...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
4,"How can startups effectively hire, manage, pro...",[Contents\nTHE PMARCA GUIDE TO STARTUPS\nPart ...,"Startups can effectively hire, manage, promote...",simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
5,How can combining an MBA with another degree e...,[degree with an MBA. I’ll hire as many of thos...,Combining an MBA with another degree can enhan...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
6,How does evolutionary behavior influence the D...,[Four: Doubt-Avoidance Tendency\nThe brain of ...,Evolutionary behavior influences the Doubt-Avo...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
7,"What should you do when VCs say ""no"" to fundin...","[Part 2: When the VCs say ""no""\nThis post is a...","When VCs say 'no' to funding your startup, it ...",simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
8,How can lack of curiosity be a danger to a sta...,"[cated email desktop soaware, but rather the u...",Lack of curiosity can be a huge danger to a st...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
9,How does not keeping a schedule benefit indivi...,"[By not keeping a schedule, I mean: refuse to ...",The answer to given question is not present in...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True


#### ❓ Question #3:

`{simple: 0.5, reasoning: 0.25, multi_context: 0.25}`

What exactly does this mapping refer to?
- 50% of the questions that are generated should be simple and easy to answer based on the source content.
- 25% of the questions should require some degree of reasoning from the LLM to be answered effectively.
- 25% of the questions should require the LLM to incorporate multiple pieces of context from the source data to be answered correctly.

> NOTE: Check out the Ragas documentation on this generation process [here](https://docs.ragas.io/en/stable/concepts/testset_generation.html).

Let's look at the output and see what we can learn about it!

In [33]:
testset.test_data[0]

DataRow(question='How do the roles and responsibilities of executives in big companies differ from those in startups?', contexts=['from scratch. This is a sharp diWerence from many big\ncompany executives, who can spend their entire careers\nrunning organizations other people built — oaen years or\ndecades earlier.\n•\nBe a primary individual contributor– a startup executive\nmust “roll up her sleeves” and produce output herself. There\nare no shortage of critical things to be done at a startup, and\nan executive who cannot personally produce while\nsimultaneously building and running her organization\ntypically will not last long. Again, this is a sharp diWerence\nfrom many big companies, where executives oaen serve\nmore as administrators and bureaucrats.\n•\nBe a team player– a startup executive must take personal\nresponsibility for her relationships with her peers and people\nthroughout the startup, in all functions and at all levels. Big\ncompanies can oaen tolerate internal riva

### Generating Responses with RAG Pipeline

Now that we have some QC pairs, and some ground truths, let's evaluate our RAG pipeline using Ragas.

The process is, again, quite straightforward - thanks to Ragas and LangChain!

Let's start by extracting our questions and ground truths from our create testset.

We can start by converting our test dataset into a Pandas DataFrame.

In [34]:
test_df = testset.to_pandas()

In [35]:
test_df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,How do the roles and responsibilities of execu...,[from scratch. This is a sharp diWerence from ...,The roles and responsibilities of executives i...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
1,How can internal confusion impact a startup's ...,"[developing a large market, as opposed to Xght...",Internal confusion can impact a startup's stra...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
2,How should founders approach market projection...,[Rethink this one from the ground up. Lots of ...,Founders should avoid saying they have no comp...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
3,What is Rachleff's Law of Startup Success and ...,"[And when you have a great market, the team is...",Rachleff's Law of Startup Success states that ...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
4,"How can startups effectively hire, manage, pro...",[Contents\nTHE PMARCA GUIDE TO STARTUPS\nPart ...,"Startups can effectively hire, manage, promote...",simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
5,How can combining an MBA with another degree e...,[degree with an MBA. I’ll hire as many of thos...,Combining an MBA with another degree can enhan...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
6,How does evolutionary behavior influence the D...,[Four: Doubt-Avoidance Tendency\nThe brain of ...,Evolutionary behavior influences the Doubt-Avo...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
7,"What should you do when VCs say ""no"" to fundin...","[Part 2: When the VCs say ""no""\nThis post is a...","When VCs say 'no' to funding your startup, it ...",simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
8,How can lack of curiosity be a danger to a sta...,"[cated email desktop soaware, but rather the u...",Lack of curiosity can be a huge danger to a st...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
9,How does not keeping a schedule benefit indivi...,"[By not keeping a schedule, I mean: refuse to ...",The answer to given question is not present in...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True


In [36]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

Now we'll generate responses using our RAG pipeline using the questions we've generated - we'll also need to collect our retrieved contexts for each question.

We'll do this in a simple loop to see exactly what's happening!

In [37]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

Now we can wrap our information in a Hugging Face dataset for use in the Ragas library.

In [38]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's take a peek and see what that looks like!

In [39]:
response_dataset[0]

{'question': 'How do the roles and responsibilities of executives in big companies differ from those in startups?',
 'answer': 'Executives in big companies require different skill sets compared to executives in startups.',
 'contexts': ['are very Even great big company executives frequently\nhave no idea what to do once they arrive at a startup.\n•',
  'will succeed without you.\n•\nBeware hiring a big company executive for a startup.The\nexecutive skill sets required for a big company vs a startup',
  'how to hire, manage, promote, and Xre executives in a startup\nbased on my personal observations and experiences.',
  'up and comer at a midsized company but wants a shot at\nbeing a primary executive at a startup can be a great catch.\n•'],
 'ground_truth': 'The roles and responsibilities of executives in big companies often involve running organizations built by others, serving more as administrators and bureaucrats, and tolerating internal rivalries and warfare. In contrast, startup 

# 🤝 Breakout Room Part #2

## Task 1: Evaluating our Pipeline with Ragas

Now that we have our response dataset - we can finally get into the "meat" of Ragas - evaluation!

First, we'll import the desired metrics, then we can use them to evaluate our created dataset!

Check out the specific metrics we'll be using in the Ragas documentation:

- [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html)
- [Answer Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html)
- [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html)
- [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html)
- [Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html)

See the accompanied presentation for more in-depth explanations about each of the metrics!

In [40]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

All that's left to do is call "evaluate" and away we go!

In [43]:
results = evaluate(response_dataset, metrics, raise_exceptions=False)

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

Runner in Executor raised an exception
Traceback (most recent call last):
  File "/opt/miniconda3/lib/python3.12/site-packages/openai/_base_client.py", line 1589, in _request
    response.raise_for_status()
  File "/opt/miniconda3/lib/python3.12/site-packages/httpx/_models.py", line 761, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Client error '429 Too Many Requests' for url 'https://api.openai.com/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/miniconda3/lib/python3.12/asyncio/tasks.py", line 520, in wait_for
    return await fut
           ^^^^^^^^^
  File "/opt/miniconda3/lib/python3.12/site-packages/ragas/metrics/_context_precision.py", line 161, in _ascore
    results = await self.llm.generate(
              ^^^^^^^^^^^^^^^^^^^^^^^^
  Fil

In [44]:
results

{'faithfulness': 0.6974, 'answer_relevancy': 0.8978, 'context_recall': 0.5972, 'context_precision': 0.7191, 'answer_correctness': 0.4019}

In [45]:
results_df = results.to_pandas()
results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,How do the roles and responsibilities of execu...,Executives in big companies require different ...,[are very Even great big company executives fr...,The roles and responsibilities of executives i...,1.000000,0.957546,0.000000,0.500000,NaN
1,How can internal confusion impact a startup's ...,Internal confusion can impact a startup's stra...,[ing about competitors.\nI see two destructive...,Internal confusion can impact a startup's stra...,0.000000,0.998546,NaN,0.750000,0.455320
2,How should founders approach market projection...,Founders should approach market projections ca...,[ularly true for startups that have not yet ac...,Founders should avoid saying they have no comp...,1.000000,0.969808,NaN,1.000000,0.519414
3,What is Rachleff's Law of Startup Success and ...,Rachleff's Law of Startup Success emphasizes t...,[Law of Startup Success:\nT\nThe #1 compan\nhe...,Rachleff's Law of Startup Success states that ...,0.666667,0.950218,1.000000,0.583333,0.579109
4,"How can startups effectively hire, manage, pro...","Startups can effectively hire, manage, promote...","[how to hire, manage, promote, and Xre executi...","Startups can effectively hire, manage, promote...",0.000000,0.934386,1.000000,1.000000,NaN
5,How can combining an MBA with another degree e...,Combining an MBA with another degree can be a ...,[bly can.\nAn MBA plus a law degree can be a g...,Combining an MBA with another degree can enhan...,1.000000,0.872958,NaN,1.000000,NaN
6,How does evolutionary behavior influence the D...,Evolutionary behavior influences the Doubt-Avo...,[Four: Doubt-Avoidance Tendency\nThe brain of ...,Evolutionary behavior influences the Doubt-Avo...,0.500000,0.949224,NaN,1.000000,NaN
7,"What should you do when VCs say ""no"" to fundin...",Meeting with more VCs after a bunch have said ...,"[Part 2: When the VCs say ""no""\nThis post is a...","When VCs say 'no' to funding your startup, it ...",1.000000,0.937992,NaN,0.805556,NaN
8,How can lack of curiosity be a danger to a sta...,Lack of curiosity can be a huge danger to a st...,[the importance of hiring curious people — is ...,Lack of curiosity can be a huge danger to a st...,1.000000,0.852250,NaN,1.000000,0.405544
9,How does not keeping a schedule benefit indivi...,Not keeping a schedule benefits individuals in...,"[By not keeping a schedule, I mean: refuse to ...",The answer to given question is not present in...,0.500000,1.000000,0.000000,0.000000,0.179815


## Task 2: Making Adjustments to our RAG Pipeline

Now that we have established a baseline - we can see how any changes impact our pipeline's performance!

Let's modify our retriever and see how that impacts our Ragas metrics!

> NOTE: MultiQueryRetriever is expanded on [here](https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever) but for now, the implementation is not important to our lesson!

In [46]:
from langchain.retrievers import MultiQueryRetriever

advanced_retriever = MultiQueryRetriever.from_llm(retriever=retriever, llm=primary_qa_llm)

We'll also re-create our RAG pipeline using the abstractions that come packaged with LangChain v0.1.0!

First, let's create a chain to "stuff" our documents into our context!

In [47]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(primary_qa_llm, retrieval_qa_prompt)

Next, we'll create the retrieval chain!

In [48]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(advanced_retriever, document_chain)

In [49]:
response = retrieval_chain.invoke({"input": "Who is Taylor Swift fueding with?"})

Task exception was never retrieved
future: <Task finished name='Task-8353' coro=<as_completed.<locals>.sema_coro() done, defined at /opt/miniconda3/lib/python3.12/site-packages/ragas/executor.py:35> exception=TimeoutError()>
Traceback (most recent call last):
  File "/opt/miniconda3/lib/python3.12/asyncio/tasks.py", line 520, in wait_for
    return await fut
           ^^^^^^^^^
  File "/opt/miniconda3/lib/python3.12/site-packages/ragas/metrics/_context_recall.py", line 169, in _ascore
    results = await self.llm.generate(
              ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/lib/python3.12/site-packages/ragas/llms/base.py", line 93, in generate
    return await agenerate_text_with_retry(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/lib/python3.12/site-packages/tenacity/_asyncio.py", line 142, in async_wrapped
    return await fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/lib/python3.12/site-packages/tenacity/_asyncio.

In [50]:
print(response["answer"])

I'm sorry, but I cannot provide an answer to your question based on the context provided.


In [51]:
response = retrieval_chain.invoke({"input": "Why are they fueding?"})

In [52]:
print(response["answer"])

The feud mentioned in the context seems to be between two groups, with one group having raised a lot of money while the other did not. The reason for the feud is not explicitly stated in the provided text.


Well, just from those responses this chain *feels* better - but lets see how it performs on our eval!

Let's do the same process we did before to collect our pipeline's contexts and answers.

In [53]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

Now we can convert this into a dataset, just like we did before.

In [54]:
response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's evaluate on the same metrics we did for the first pipeline and see how it does!

In [56]:
advanced_retrieval_results = evaluate(response_dataset_advanced_retrieval, metrics, raise_exceptions=False)

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

Runner in Executor raised an exception
Traceback (most recent call last):
  File "/opt/miniconda3/lib/python3.12/asyncio/tasks.py", line 520, in wait_for
    return await fut
           ^^^^^^^^^
  File "/opt/miniconda3/lib/python3.12/site-packages/ragas/metrics/_context_precision.py", line 161, in _ascore
    results = await self.llm.generate(
              ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/lib/python3.12/site-packages/ragas/llms/base.py", line 93, in generate
    return await agenerate_text_with_retry(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/lib/python3.12/site-packages/tenacity/_asyncio.py", line 142, in async_wrapped
    return await fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/lib/python3.12/site-packages/tenacity/_asyncio.py", line 68, in __call__
    await self.sleep(do)
  File "/opt/miniconda3/lib/python3.12/asyncio/tasks.py", line 665, in sleep
    return await future
           ^^^^^^^^^^^^
  File 

In [57]:
advanced_retrieval_results_df = advanced_retrieval_results.to_pandas()
advanced_retrieval_results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,How do the roles and responsibilities of execu...,The executive skill sets required for a big co...,[are very Even great big company executives fr...,The roles and responsibilities of executives i...,1.000000,0.959817,0.000000,0.250000,NaN
1,How can internal confusion impact a startup's ...,Internal confusion within a startup can impact...,[ing about competitors.\nI see two destructive...,Internal confusion can impact a startup's stra...,0.000000,0.995979,0.000000,NaN,0.426376
2,How should founders approach market projection...,Founders should approach market projections in...,"[founding team with a business person, you hav...",Founders should avoid saying they have no comp...,0.714286,0.969808,NaN,1.000000,NaN
3,What is Rachleff's Law of Startup Success and ...,Rachleff's Law of Startup Success emphasizes t...,[Law of Startup Success:\nT\nThe #1 compan\nhe...,Rachleff's Law of Startup Success states that ...,1.000000,0.950218,1.000000,0.416667,NaN
4,"How can startups effectively hire, manage, pro...","Startups can effectively hire, manage, promote...","[how to hire, manage, promote, and Xre executi...","Startups can effectively hire, manage, promote...",1.000000,1.000000,NaN,NaN,NaN
5,How can combining an MBA with another degree e...,Combining an MBA with another degree can enhan...,[bly can.\nAn MBA plus a law degree can be a g...,Combining an MBA with another degree can enhan...,0.166667,1.000000,1.000000,0.833333,NaN
6,How does evolutionary behavior influence the D...,Evolutionary behavior may influence the Doubt-...,"[Tendency, what usually triggers Doubt-Avoidan...",Evolutionary behavior influences the Doubt-Avo...,0.250000,0.916201,NaN,NaN,0.721812
7,"What should you do when VCs say ""no"" to fundin...","When VCs say ""no"" to funding your startup, it ...",[Xrst place. What if you have a startup for wh...,"When VCs say 'no' to funding your startup, it ...",1.000000,0.999141,1.000000,0.810000,0.796941
8,How can lack of curiosity be a danger to a sta...,Lack of curiosity can be a danger to a startup...,[the importance of hiring curious people — is ...,Lack of curiosity can be a huge danger to a st...,1.000000,0.962270,NaN,NaN,0.560285
9,How does not keeping a schedule benefit indivi...,Not keeping a schedule can benefit individuals...,[sibilities where they can’t do that. Or maybe...,The answer to given question is not present in...,0.500000,0.993320,NaN,0.000000,0.178363


## Task 3: Evaluating our Adjusted Pipeline Against Our Baseline

Now we can compare our results and see what directional changes occured!

Let's refresh with our initial metrics.

In [ ]:
results

{'faithfulness': 0.6255, 'answer_relevancy': 0.8526, 'context_recall': 0.6500, 'context_precision': 0.7583, 'answer_correctness': 0.5520}

And see how our advanced retrieval modified our chain!

In [ ]:
advanced_retrieval_results

{'faithfulness': 0.7120, 'answer_relevancy': 0.9455, 'context_recall': 0.7417, 'context_precision': 0.7112, 'answer_correctness': 0.5529}

In [ ]:
import pandas as pd

df_original = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_comparison = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'MultiQueryRetriever with Document Stuffing'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')

df_merged['Delta'] = df_merged['MultiQueryRetriever with Document Stuffing'] - df_merged['Baseline']

df_merged

,Metric,Baseline,MultiQueryRetriever with Document Stuffing,Delta
0,faithfulness,0.625490,0.712024,0.086534
1,answer_relevancy,0.852605,0.945499,0.092894
2,context_recall,0.650000,0.741667,0.091667
3,context_precision,0.758333,0.711185,-0.047149
4,answer_correctness,0.551982,0.552914,0.000932


## Task 4: Testing OpenAI's Claim

Now that we've seen how our retriever can impact the performance of our RAG pipeline - let's see how changing our embedding model impacts performance.

####🏗️ Activity #2:

Please provide markdown, or code comments, to explain which each of the following steps are doing!

In [ ]:
new_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [ ]:
vector_store = Qdrant.from_documents(
    documents,
    new_embeddings,
    location=":memory:",
    collection_name="PMarca Blogs - TE3 - MQR",
)

In [ ]:
new_retriever = vector_store.as_retriever()

In [ ]:
new_advanced_retriever = MultiQueryRetriever.from_llm(retriever=new_retriever, llm=primary_qa_llm)

In [ ]:
new_retrieval_chain = create_retrieval_chain(new_advanced_retriever, document_chain)

In [ ]:
answers = []
contexts = []

for question in test_questions:
  response = new_retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

In [ ]:
new_response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [ ]:
new_advanced_retrieval_results = evaluate(new_response_dataset_advanced_retrieval, metrics)

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
new_advanced_retrieval_results

{'faithfulness': 0.8658, 'answer_relevancy': 0.9454, 'context_recall': 0.6792, 'context_precision': 0.8472, 'answer_correctness': 0.5055}

In [ ]:
df_baseline = pd.DataFrame(list(results.items()), columns=['Metric', 'ADA + Baseline'])
df_original = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'ADA + MQR'])
df_comparison = pd.DataFrame(list(new_advanced_retrieval_results.items()), columns=['Metric', 'TE3 + MQR'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')
df_merged = pd.merge(df_baseline, df_merged, on="Metric")

df_merged['ADA + MQR -> TE3 + MQR'] = df_merged['TE3 + MQR'] - df_merged['ADA + MQR']
df_merged['Baseline -> TE3 + MQR'] = df_merged['TE3 + MQR'] - df_merged['ADA + Baseline']

df_merged

,Metric,ADA + Baseline,ADA + MQR,TE3 + MQR,ADA + MQR -> TE3 + MQR,Baseline -> TE3 + MQR
0,faithfulness,0.625490,0.712024,0.865833,0.153810,0.240343
1,answer_relevancy,0.852605,0.945499,0.945435,-0.000064,0.092831
2,context_recall,0.650000,0.741667,0.679167,-0.062500,0.029167
3,context_precision,0.758333,0.711185,0.847174,0.135990,0.088841
4,answer_correctness,0.551982,0.552914,0.505525,-0.047388,-0.046457


####❓ Question #4:

Do you think, in your opinion, `text-embedding-3-small` is significantly better than `ada`?

## BONUS ACTIVITY: Using a Better Generator

Now that we've seen how much more effective a better Retrieval pipeline is, let's look at what impact a better(?) Generator is!

Adapt the above `TE3 + MQR` pipeline to use `GPT-4o` and compare the results below!

In [ ]:
### YOUR CODE HERE